In [1]:
import pandas as pd



In [7]:
# 读取文件
df = pd.read_csv(r'D:\Code_File\Vinda_cunzhijia\存纸架数据汇总.csv')  # 请确保文件名和路径正确

# 参数配置
HIGH_SPEED = 100    # 定义高速阈值
LOW_SPEED = 50      # 定义低速阈值
MIN_SLOPE = -5      # 下降斜率阈值(负值)
RISING_SLOPE = 5    # 上升斜率阈值(正值)

# 计算速度变化率
df['速度变化率'] = df['折叠机实际速度'].diff()

# 找出所有关键时间点
events = []
for i in range(1, len(df)):
    prev_speed = df.at[i-1, '折叠机实际速度']
    current_speed = df.at[i, '折叠机实际速度']
    slope = df.at[i, '速度变化率']
    
    # 检测开始下降点（从高速开始明显下降）
    if prev_speed >= HIGH_SPEED and slope <= MIN_SLOPE:
        events.append(('开始下降', df.at[i, '时间']))
    
    # 检测开始上升点（从低速开始明显上升）
    elif prev_speed <= LOW_SPEED and slope >= RISING_SLOPE:
        events.append(('开始上升', df.at[i, '时间']))

# 配对下降和上升时间点
time_periods = []
period_num = 1  # 新增序号计数器
i = 0
while i < len(events):
    if events[i][0] == '开始下降':
        # 寻找下一个对应的开始上升点
        for j in range(i+1, len(events)):
            if events[j][0] == '开始上升':
                time_periods.append({
                    '序号': period_num,  # 新增序号列
                    '开始时间': events[i][1],
                    '结束时间': events[j][1],
                    '持续时间': pd.to_datetime(events[j][1]) - pd.to_datetime(events[i][1]),
                    '类型': '降速周期'
                })
                period_num += 1  # 序号递增
                i = j  # 移动指针到已处理的上升点
                break
    i += 1

# 转换为DataFrame并保存
result_df = pd.DataFrame(time_periods)

# 调整列顺序，让序号作为第一列
result_df = result_df[['序号', '开始时间', '结束时间', '持续时间', '类型']]

result_df.to_csv('D:\Code_File\Vinda_cunzhijia\高低速时间段\折叠机速度周期时间段.csv', index=False, encoding='utf_8')

# 打印结果
print("已识别到的速度周期：")
print(result_df)
print(f"\n结果已保存到: 折叠机速度周期时间段.csv")


已识别到的速度周期：
      序号                 开始时间                 结束时间            持续时间    类型
0      1  2025-04-30 16:12:07  2025-04-30 16:16:06 0 days 00:03:59  降速周期
1      2  2025-04-30 16:19:19  2025-04-30 16:25:22 0 days 00:06:03  降速周期
2      3  2025-04-30 16:27:06  2025-04-30 16:28:42 0 days 00:01:36  降速周期
3      4  2025-04-30 16:30:20  2025-04-30 16:34:58 0 days 00:04:38  降速周期
4      5  2025-04-30 16:36:06  2025-04-30 16:40:25 0 days 00:04:19  降速周期
..   ...                  ...                  ...             ...   ...
115  116  2025-05-06 06:01:02  2025-05-06 06:07:27 0 days 00:06:25  降速周期
116  117  2025-05-06 06:55:22  2025-05-06 06:58:33 0 days 00:03:11  降速周期
117  118  2025-05-06 07:43:32  2025-05-06 07:48:04 0 days 00:04:32  降速周期
118  119  2025-05-06 09:37:13  2025-05-06 10:03:48 0 days 00:26:35  降速周期
119  120  2025-05-06 11:08:37  2025-05-06 11:09:59 0 days 00:01:22  降速周期

[120 rows x 5 columns]

结果已保存到: 折叠机速度周期时间段.csv
